In [2]:
import pandas as pd
import quandl
import math

In [3]:
df = quandl.get('WIKI/GOOGL')

In [4]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_percent'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100
df['day_percent'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100
df = df[['Adj. Close', 'HL_percent', 'day_percent']]

In [5]:
print df.head()

            Adj. Close  HL_percent  day_percent
Date                                           
2004-08-19   50.322842    8.441017     0.324968
2004-08-20   54.322689    8.537313     7.227007
2004-08-23   54.869377    4.062357    -1.227880
2004-08-24   52.597363    7.753210    -5.726357
2004-08-25   53.164113    3.966115     1.183658


In [7]:
forecast_col = 'Adj. Close'
df.fillna(-9999999, inplace=True)
forecast_days = int(math.ceil(0.01*len(df)))
df['label'] = df[forecast_col].shift(-forecast_days)
df.dropna(inplace=True)
print df.head()

            Adj. Close  HL_percent  day_percent      label
Date                                                      
2004-10-06   68.752232    1.801471    -0.428561  82.805613
2004-10-07   69.639972    2.438667     1.676919  84.019360
2004-10-08   69.078238    1.941322    -0.720825  87.650569
2004-10-11   67.839414    3.742996    -1.277279  89.972737
2004-10-12   68.912727    3.155922     2.163730  90.805307
